# Lab 7: Zonal Operation and Spatial Interpolation

Objectives of the lab:

1. Practise to use zonal operation to aggregate raster data in polygons
2. Use spatial interpolation methods to fill a surface from sampled points.

You can use the data downloaded for Lab 6. Otherwise, you can re-download the data from [here](https://drive.google.com/open?id=14tO8vbIGmQs-xLH3rajEtQjDXjnYU6vB).

**Submission**: submit the Jupyter Notebook file (.ipynb) containing the completed code and maps through Laulima by **Monday, April 22nd**.

Before the lab, please install the following packages in Anaconda Prompt as an administrator user:

 - conda install rasterstats

# Task 1: Zonal Operation

Zonal operation is a often used method to aggregate raster pixels into discrete zones (polygons). This method can be used to related raster data and data reported in polygon boundaries (e.g. census data). In this task, we will aggregate a land cover raster into census tracts to find out the most urbanized census tracts in Oahu.

First, complete the following code to read census tract polygons in CensusTract_Oahu.shp.

In [ ]:
# import the needed package
import geopandas

# read the shapefile as a geodataframe and save it in censustract.
# It would be convenient to set the default working directory to the folder containing all the data.
# You can refer to Lab 6 for how to set working directory
censustract = geopandas.read_file("C:/Users/yi/Documents/UH_work/Teaching/GEOG389/labs/lab6_data/CensusTract_Oahu.shp")

Read the land cover raster using the rasterio package

In [ ]:
import rasterio
lc = rasterio.open("C:/Users/yi/Documents/UH_work/Teaching/GEOG389/labs/lab6_data/hi_oahu_2011_ccap_hr_land_cover20140619.img")

You can view the metadata of `lc`. 

In [ ]:
lc.meta

You need to ensure the raster and polygon datasets are in the same coordinate system before the zonal operation.

View the coordinate systems of the two datasets.

In [ ]:
censustract.crs

In [ ]:
lc.crs

The epsg numbers are the unique IDs of coordinate systems. You search the specific coordinate systems using the epsg numbers in [https://www.epsg-registry.org/](https://www.epsg-registry.org/).

As displayed above, the different epsg values indicate that the two datasets are in different coordinate systems. 
- epgs:4269 is a geographic coordinate system based on lat/long and the NAD83 datum.
- epsg:32604 is a projected coordinate system for UTM zone 4N (the UTM zone of Oahu). Usually a raster dataset is project in a projected coordinate system.

Next, we need to convert the coordinate system of censustract to UTM zone 4N, which is the coordinate system of the land cover raster.

Usually, it's betterr to convert the coordinate system of the vector instead of raster. Converting the coordinate system of raster data will go through resampling, which introduce uncertainty in the resampled raster.

In [ ]:
#import os
#os.environ['PROJ_LIB']=r"C:\ProgramData\Anaconda3\Library\share"

censustract = censustract.to_crs(lc.crs.data)

After coordinate conversion, the two data layers can be plotted in the same map

In [ ]:
import matplotlib.pyplot as plt
from rasterio.plot import show

f, ax = plt.subplots(1, figsize=(9, 9))

show((lc, 1),ax=ax)

censustract.plot(ax=ax, facecolor='None', edgecolor='white', linewidth=1)

Next, we will use zonal operation to aggregate raster pixels in the census tracts. 

Extract the 1st band (also the only band) of the raster and store it in a numpy array. The extracted array is a 2D matrix of numbers (not spatially projected).

In [ ]:
array = lc.read(1)

Print the unique values in the array.

In [ ]:
import numpy as np
np.unique(array)

These values represent different land cover type. The land cover types of these values can be found in [here](https://coast.noaa.gov/data/digitalcoast/pdf/ccap-class-scheme-regional.pdf).

For instance, values from 2 to 5 represent developed land with different densities. In this task, we consider all the developed land types are urban land.

In this task, we will calculate the ratios of urban land in different census tracts. The strategy is counting the numbers of urban pixels and total pixels in each census tract. Then, urban ratio is calculated by dividing the urban pixel count by the total pixel count in each census tract.

To do the division, both the counts of urban pixels and counts of total pixels need to be aggregated in census tracts.

First, reclassify all urban pixels (value from 2 to 5) to 1. We cansider all types of developed land as urban land.

In [ ]:
# reclassify pixels from 2 to 5 to 1.
array[np.where((array >= 2)&(array<=5))] = 1

Reclassify non-urban pixels (value above 5) to 0.

In [ ]:
array[np.where(array > 5)] = 0

Plot the reclassified raster

In [ ]:
# plot the reclassified raster
f, ax = plt.subplots(1, figsize=(9, 9))
imgplot = plt.imshow(array)

Then, using zonal operation ('zonal_stats') to aggregate the reclassified binary raster (0: non-urban, 1: urban) into census tracts. Here, the `stats` parameter defines the function(s) of aggregation. Here, the sum (sum of urban pixels) and count (total pixels) are aggregated. `affine` defines the bounding box and cellsize of the raster.

In [ ]:
from rasterstats import zonal_stats
zs_ct = zonal_stats(censustract, array, affine=lc.transform, stats=['count','sum'])

You can also aggregate pixels using mean, max, min, majority, etc. The complete description of the `zonal_states` function can be found in [https://pythonhosted.org/rasterstats/manual.html].

Print `zs_ct` to see what's inside.

In [ ]:
zs_ct

convert `zs_ct` from a 2D list to a data frame.

In [ ]:
import pandas as pd
zs_ct_df = pd.DataFrame(zs_ct)

Preview the zs_ct_df

### Question 1: Please calculate the ratios of urban pixels and store the ratios in a new column `urban_ratio`.

When calculating the ratio, think about which column should be the numerator and which to be the denominator.

In [ ]:
zs_ct_df['urban_ratio'] = zs_ct_df['sum']/zs_ct_df['count']

Preview `zs_ct_df` to check if the calculation is done appropriately.

A division calculation may lead to infinite numbers (coded as `inf` in Python) if the denominator is 0. Many mapping functions cannot deal with inf. So it is better to eliminate them.

Convert `infs` to `nans`.

In [ ]:
zs_ct_df['urban_ratio'][zs_ct_df['urban_ratio']==np.inf]=np.nan

Next, we will join the urban ratios in `zs_ct_df` back to `censustract`. 

As there is no GEOID in `zs_ct_df`, we cannot do an attribute join using keys in the two tables. As `zs_ct_df` was converted from `zs_ct` (the spatial join output), the rows in 'zs_ct_df' are in the same order as the rows in `censustract`. Thus, we can simply horizontally combine `zs_ct_df` with `censustract`.

![](images/fig18.JPG)

Horizontally combine `censustract` and `zs_ct_df` into `censustract2`.

In [ ]:
censustract2 = geopandas.GeoDataFrame(pd.concat([censustract, zs_ct_df], axis=1, ignore_index= False))

Preview `censustract2` to see if the combination is successful.

### Question 2: Create a choropleth map to display the ratos of urban area in the census tracts.

Tip: you can refer to Lab 5 for how to create the map.

Observe the map and trying to identify the most urbanized (highest urban ratio) areas.

# Task 2: Spatial Interpolation

Spatial interpolation can be used to fill the values in a continuous surface from point observations/samples/measurements. Spatial interpolation is based on spatial autocorrelation: closer locations have similar values.

Commonly used spatial interpolation methods include Thiessen/Voronoi polygons, Triangular Irregular Network (TIN), Inverse Distance Weighting, Spline (polynomial) or Kriging. 

In this exercise, we will interpolate ozone measurements in a number of observation stations in California using point-based temperature measurements. We will use the Voronoi polygon (nearest), TIN (linear), and polynomial (cubic) methods in the `scipy` package to do the interpolation. In reality, the krigin method is most popular. However, we don't do it in this lab due to the compuational intensity and coding complexity in Python. However, krigin is supported by most GIS software including ArcGIS and QGIS.

Complete the following code to read the temperature data in _temp.shp_.

In [ ]:
ozone = geopandas.read_file("ozone.shp")

Preview `ozone` to see what is inside. The column _ozone_ is the variable we will interpolate.

### Question 3: Please complete the following code to plot the station points on a base map.

You can use different markersizes to represent the ozone values (markersize=ozone['ozone']*10).


In [ ]:
from mpl_toolkits.basemap import Basemap
# You need to replace the path with the folder of Anaconda in your computer
os.environ['PROJ_LIB'] = 'C:/ProgramData/Anaconda3/Library/share/'

f, ax1 = plt.subplots(1, figsize=(15, 10))

# Get the bounding box of ozone points
xlim=([min(ozone.LON),max(ozone.LON)])
ylim=([min(ozone.LAT),max(ozone.LAT)])

# plot a basemap zooming into the boundingbox of the zone points.
map = Basemap(llcrnrlon=xlim[0],llcrnrlat=ylim[0],urcrnrlon=xlim[1],urcrnrlat=ylim[1], epsg=4269, ax=ax1)
#https://www.bdmweather.com/2018/04/python-m-arcgisimage-basemap-options/

map.arcgisimage(service='World_Topo_Map', xpixels = 2000, verbose= True)

# plot the stations. The dot size indicates the ozone value.
### input code here to plot the station points ####


You can observe the various ozone values at the observation stations. Ozone values in the black areas can be estimated using spatial interpolation

Next, we will use functions in the `scipy` package (default in Python 3.x) to interpolate temperature. In this task, we will use the nearest (Voronoi polygon), linear (TIN) and cubic (spline/polynomial) methods to interpolate.

`ozone` was originally in a geographic coordinate system (lat&lon). 

For spatial interpolation, we need convert the coordinate system of `ozone` to a projected coordinate system for the distance calculation. 

In [ ]:
ozone = ozone.to_crs({'init': 'epsg:32611'})

To use the interpolation functions in the `scipy` package, the input coordinates (x, y) need to be organized in a 2D numpy array. The temperature at the points need to be stored in a 1D array.

The following code is to extract the x, y coordinates from the geometry of `temp` and save x and y into a 2D array.

In [ ]:
xy = pd.concat([ozone.geometry.x, ozone.geometry.y],axis=1)
xy = np.array(xy)

Convert the ozone values (the variable to be interpolated) into a 1D array.

In [ ]:
ozone_a=np.array(ozone.ozone)

Now, the input data (x,y, and temperature) are stored in a 2D and 1D array respectively, which are ready for use.

Next, we create a 2D raster surface for the output surface. Spatial interpolation is estimating values in every location (i.e. pixel) in the raster space. 

As the points are located in southern Clalifornia. It is reasonable to interpolate an area cover the points locations. The following code get the bounding box (i.e. the range of x and y) of the points.

In [ ]:
bx = ([ozone.total_bounds[0],  ozone.total_bounds[2]])
by = ([ozone.total_bounds[1],  ozone.total_bounds[3]])

Create 2 2D arrays to store the x,y coordinates of pixels in the raster. The raster covering the bounding box of the `temp` points. The raster is 1000 rows * 1000 columns

In [ ]:
grid_x,grid_y = np.mgrid[bx[0]:bx[1]:1000j, by[0]:by[1]:1000j]

Interpolate the ozone surface using Voronoi (Thiessen) polygons.

In [ ]:
from scipy.interpolate import griddata
grid_z0 = griddata(xy, ozone_a, (grid_x, grid_y), method='nearest')

The interpolated surface (Voronoi polygons) is stored in `grid_z0`. The following code displays the interpolation result.

In [ ]:
import matplotlib.pyplot as plt


# set the plot size
plt.figure(figsize = (15,10))

# plot the the interpolated surface and get the axes (canvas) of the plot
plt.imshow(grid_z0.T, cmap='Blues',extent=(bx[0],bx[1],by[0],by[1]), origin='lower')

# plot the station points. Dot size indicate the ozone value
plt.scatter(xy[:,0], xy[:,1], s=ozone_a*10, c='red', marker='o')

plt.title('Nearest (Voronoi Polygons)')
plt.show()

### Question 4: Please interpolate the ozone values using the linear (TIN) and cubic (polynomial) method.

Tip: you can replace the method parameter from _nearest_ to _linear (TIN)_ and _cubic_.

In [ ]:
# Interpolate using the linear (TIN) method



In [ ]:
# Interpolate using the cubic (polynomial) method



****

# Other Conversion Operation

Lab 6 and 7 applied spatial join (point to polygon), zonal operation (raster to polygon) and spatial interpolation (point to surface (raster)). Tutorials of the other spatial data conversion introduced in class can be found from:

1. [Raster Resampling](https://rasterio.readthedocs.io/en/stable/topics/resampling.html)
2. [Raster value to point](https://www.earthdatascience.org/courses/earth-analytics-python/lidar-remote-sensing-uncertainty/extract-data-from-raster/)
3. [Rasterize polygon](https://rasterio.readthedocs.io/en/stable/api/rasterio.features.html)